In [ ]:
import geopandas as gpd
from shapely.ops import nearest_points
from tqdm import tqdm

elec_nodes = gpd.read_file('../data/spatial/infrasim-network/nodes.shp')
water_nodes = gpd.read_file('../data/water/merged_water_assets.shp')

In [ ]:
gpd1 = elec_nodes.loc[elec_nodes.subtype.isin(['pole','substation'])].reset_index(drop=True).copy()
gpd2 = water_nodes.loc[water_nodes.linkage == 'True'].reset_index(drop=True).copy()

pts3 = gpd1.geometry.unary_union

gpd2['elec_asset'] = ''

count=0
for i, row in tqdm(gpd2.iterrows(),total=gpd2.shape[0]):
    nearest_elec_asset = nearest_points(row.geometry, pts3)[1]
    gpd2.loc[i,'elec_asset'] = gpd1.loc[gpd1.geometry.within(nearest_elec_asset),'id'].values[0]
    count=count+1

In [ ]:
water_nodes['elec_asset'] = water_nodes['node_id'].map(\
                    gpd2[['node_id','elec_asset']].set_index('node_id')['elec_asset'].to_dict())



water_nodes.to_file(driver='ESRI Shapefile',filename='../_tmp/elec_nodes_mapped.shp')